# Projected Expected Average Statistics 
## Simple Projection System

The end goal of this project is to determine how players shoot when they are over- or under-performing, measured by their holistic performance on the court. <br>
We are measuring their holsitic performance on the court by comparing their actual statistics to the expected statistic.<br>
Which begs the question: "How can we accurately predict expected averages?"<br>
<br>
This project will utilize data that uses the simplest method possible to predict expected averages, the Simple Projection System (SPS). The SPS was chosen because these values themselves are going to be used in <br> calculations for a subsequent predictive model, and I wasn't comfortable building a novel predictive model accurate or precise enough to withstand compounding error analysis. Such a model itself could be the focus of <br> its own research (I can't envision Vegas or Macao releasing their predictive models for academic research), and as such, falls outside the scope of this paper. 
<br>
The SPS data was taken from basketball-reference.com - the link to the SPS explanation can be found here (https://www.basketball-reference.com/about/projections.html), but I'll give a brief outline of the process below.
<br>
1. The model takes the last three years of data into account, with the most recent year given a weight of 6, the second a weight of 3, and the third a weight of 1. <br>
2. The model then takes the weighted sum of minutes played <br>
3. and the weighted sum of the target statistic. <br>
4. Then, the model calculates the weighted sum of a league-average player scaled to the target player's minutes played, then multiplied by 1000. <br>
5. Calculating the player's expected statistic for 36 minutes is done as follows: <br>
<br>
(step 3 value + step 4 value) / (step 2 value + 1000) * 36 
<br> <br>
6. Finally, there is an Age Adjustment made - this takes into account that most players peak around 28 years of age. <br>
If the player is younger than 28, then the adjustment value is (age - 28) * 0.004. If they are older, then it's (28 - age) * 0.002. <br> <br>

Some additional notes, straight from BR:<br>
A few more notes regarding the SPS:<br>

For the following categories the sign of the age adjustment is reversed: field goals missed, 3-point field goals missed, free throws missed, turnovers, and personal fouls.<br>
For the shooting categories, shots missed are projected rather than shots attempted. Projected shots attempted are then computed by adding projected shots made and projected shots missed.<br>
Projected points are computed using projected field goals made, projected 3-point field goals made, and projected free throws made.<br>

### SPS projections

In [156]:
# importing necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from sklearn.metrics import mean_squared_error
from math import sqrt

In [157]:
marcels_1718 = pd.read_csv('/Users/seandiehl/Downloads/1718marcels.csv')
marcels_1819 = pd.read_csv('/Users/seandiehl/Downloads/marcels1819.csv')
pd.set_option('display.max_columns', None)

In [161]:
marcels_1819.head()

,Rk,Player,Type,FG,FGA,3P,3PA,FT,FTA,ORB,TRB,AST,STL,BLK,TOV,PF,PTS,FG%,3P%,FT%
0,1,Álex Abrines\abrinal01,Projected,4.2,10.2,2.8,7.1,1.5,1.7,0.8,3.7,1.3,1.2,0.3,1.0,3.8,12.5,0.407,0.385,0.852
1,1,Álex Abrines\abrinal01,Actual,3.4,9.6,2.5,7.8,0.7,0.8,0.3,2.9,1.2,1.0,0.4,0.9,3.2,10.1,0.357,0.323,0.923
2,2,Quincy Acy\acyqu01,Projected,3.9,10.1,2.4,6.7,1.7,2.1,1.2,6.9,1.6,0.9,0.8,1.6,3.9,11.9,0.389,0.358,0.786
3,2,Quincy Acy\acyqu01,Actual,1.2,5.3,0.6,4.4,2.0,2.9,0.9,7.3,2.3,0.3,1.2,1.2,7.0,5.0,0.222,0.133,0.700
4,3,Steven Adams\adamsst01,Projected,6.1,10.1,0.1,0.2,2.3,4.0,4.9,9.7,1.4,1.3,1.2,1.9,3.1,14.7,0.609,0.327,0.588


In [164]:
projections = marcels_1819.loc[marcels_1819['Type']=='Projected']

projections.head()


,Rk,Player,Type,FG,FGA,3P,3PA,FT,FTA,ORB,TRB,AST,STL,BLK,TOV,PF,PTS,FG%,3P%,FT%
0,1,Álex Abrines\abrinal01,Projected,4.2,10.2,2.8,7.1,1.5,1.7,0.8,3.7,1.3,1.2,0.3,1.0,3.8,12.5,0.407,0.385,0.852
2,2,Quincy Acy\acyqu01,Projected,3.9,10.1,2.4,6.7,1.7,2.1,1.2,6.9,1.6,0.9,0.8,1.6,3.9,11.9,0.389,0.358,0.786
4,3,Steven Adams\adamsst01,Projected,6.1,10.1,0.1,0.2,2.3,4.0,4.9,9.7,1.4,1.3,1.2,1.9,3.1,14.7,0.609,0.327,0.588
6,4,Bam Adebayo\adebaba01,Projected,4.9,9.4,0.2,0.6,3.4,4.6,3.0,9.9,2.8,0.9,1.1,1.7,3.5,13.3,0.518,0.282,0.735
8,5,LaMarcus Aldridge\aldrila01,Projected,8.9,18.0,0.4,1.2,4.2,5.1,3.1,8.7,2.2,0.7,1.3,1.6,2.4,22.4,0.495,0.317,0.829


In [170]:
projections['PlayerID'] = projections['Player'].str.split('''\\''').str[1]
projections['Name'] = projections['Player'].str.split('''\\''').str[0]
projections.head()

/var/folders/zm/h6ndcwpn4h72pqxjr10sfd680000gn/T/ipykernel_93757/2871419775.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  projections['PlayerID'] = projections['Player'].str.split('''\\''').str[1]
/var/folders/zm/h6ndcwpn4h72pqxjr10sfd680000gn/T/ipykernel_93757/2871419775.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  projections['Name'] = projections['Player'].str.split('''\\''').str[0]


,Rk,Player,Type,FG,FGA,3P,3PA,FT,FTA,ORB,TRB,AST,STL,BLK,TOV,PF,PTS,FG%,3P%,FT%,PlayerID,Name
0,1,Álex Abrines\abrinal01,Projected,4.2,10.2,2.8,7.1,1.5,1.7,0.8,3.7,1.3,1.2,0.3,1.0,3.8,12.5,0.407,0.385,0.852,abrinal01,Álex Abrines
2,2,Quincy Acy\acyqu01,Projected,3.9,10.1,2.4,6.7,1.7,2.1,1.2,6.9,1.6,0.9,0.8,1.6,3.9,11.9,0.389,0.358,0.786,acyqu01,Quincy Acy
4,3,Steven Adams\adamsst01,Projected,6.1,10.1,0.1,0.2,2.3,4.0,4.9,9.7,1.4,1.3,1.2,1.9,3.1,14.7,0.609,0.327,0.588,adamsst01,Steven Adams
6,4,Bam Adebayo\adebaba01,Projected,4.9,9.4,0.2,0.6,3.4,4.6,3.0,9.9,2.8,0.9,1.1,1.7,3.5,13.3,0.518,0.282,0.735,adebaba01,Bam Adebayo
8,5,LaMarcus Aldridge\aldrila01,Projected,8.9,18.0,0.4,1.2,4.2,5.1,3.1,8.7,2.2,0.7,1.3,1.6,2.4,22.4,0.495,0.317,0.829,aldrila01,LaMarcus Aldridge


In [172]:
projections_per_sec = projections.apply(lambda x: (x / 2160) if x.name in ['FG', 'FGA', '3P', '3PA', 'FT', 'FTA', 'ORB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS'] else x)

projections_per_sec.head()

,Rk,Player,Type,FG,FGA,3P,3PA,FT,FTA,ORB,TRB,AST,STL,BLK,TOV,PF,PTS,FG%,3P%,FT%,PlayerID,Name
0,1,Álex Abrines\abrinal01,Projected,0.001944,0.004722,0.001296,0.003287,0.000694,0.000787,0.000370,0.001713,0.000602,0.000556,0.000139,0.000463,0.001759,0.005787,0.407,0.385,0.852,abrinal01,Álex Abrines
2,2,Quincy Acy\acyqu01,Projected,0.001806,0.004676,0.001111,0.003102,0.000787,0.000972,0.000556,0.003194,0.000741,0.000417,0.000370,0.000741,0.001806,0.005509,0.389,0.358,0.786,acyqu01,Quincy Acy
4,3,Steven Adams\adamsst01,Projected,0.002824,0.004676,0.000046,0.000093,0.001065,0.001852,0.002269,0.004491,0.000648,0.000602,0.000556,0.000880,0.001435,0.006806,0.609,0.327,0.588,adamsst01,Steven Adams
6,4,Bam Adebayo\adebaba01,Projected,0.002269,0.004352,0.000093,0.000278,0.001574,0.002130,0.001389,0.004583,0.001296,0.000417,0.000509,0.000787,0.001620,0.006157,0.518,0.282,0.735,adebaba01,Bam Adebayo
8,5,LaMarcus Aldridge\aldrila01,Projected,0.004120,0.008333,0.000185,0.000556,0.001944,0.002361,0.001435,0.004028,0.001019,0.000324,0.000602,0.000741,0.001111,0.010370,0.495,0.317,0.829,aldrila01,LaMarcus Aldridge
